In [3]:
scope.dis()
target.dis()
exit()

In [2]:
import chipwhisperer as cw
import sys
sys.path.append("../imports")
from helpers import *
from scheme import Scheme
from taskers import *

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()
scope.adc.samples=56

In [13]:

#load(scope, "xor", None)
scope.adc.trig_count 
target.flush()
scope.arm()
print(get_bytes([0 for i in range(64)]))
target.simpleserial_write('p', get_bytes([0 for i in range(64)]))
ret = scope.capture()
if ret:
    print("ERROR")
traces_ = scope.get_last_trace()
db = target.simpleserial_read('r', 5)
print(scope.adc.trig_count)


bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00')
56


In [102]:
rand = [random.randrange(2) for _ in range(16)]
c = list(rand)
for i in range(2):
    a = c[:8]
    b = c[8:]
    a = xor(a, lshift(a,3), lshift(a,7))
    b = xor(b, lshift(a,1), lshift(a,5))
    c = a + b
    c = lshift(c,4)
rand = rand + c
print(rand)

[0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1]


In [3]:

# function
def setup():
    # runs once in the start -> prints returned
    load(scope, "xor", env)
    # print(date_str())
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    rand =[random.randrange(2) for _ in range(32)]
    if op=="18":
        p_temp = p18
    elif op=="21":
        p_temp = p21
    elif op=="16_1":
        p_temp = p16_1
    elif op=="16_2":
        p_temp = p16_2
    elif op=="tv":
        pz_temp = [0]*16 + [1]*16
        p_temp = random.sample(pz_temp, 32)
    else:
        raise "error"
    return get_bytes(p_temp + rand)

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    pass

pz = [0]*16 + [1]*16
p16_1 = random.sample(pz, 32)

pz = [0]*16 + [1]*16
p16_2 = random.sample(pz, 32)

pz = [0]*14 + [1]*18
p18 = random.sample(pz, 32)

pz = [0]*11 + [1]*21
p21 = random.sample(pz, 32)

env = ENV()
# operation settings
prefix_name = "masked"
env.root = "D:/programs/traces/XOR"
env.jupyter = "mask_xor"
env.execute = execute
env.ending = ending
env.operations = operations = ["tv", "16_1", "16_2", "18", "21"] # operations list
inv = lambda x: operations.index(x)
env.subfolder = "gap4" # subfolder name
env.TRACES_NUM = 100_000 # number of traces
scope.adc.samples = 56 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 5
env.desc = ""
env.graph = {"type": "time"}
env.title = "XOR masking exeriment"
env.name = prefix_name
env.readme = f"{p16_1}\n{p16_2}\n{p18}\n{p21}"
s = Scheme() # scheme execution helper

# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()

file_names = [f"file{i}" for i in range(4)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)

i = 0

i += 1
env.name = prefix_name+f"_{i}"
env.title = "f_{p0} vs f_{p1}, mask <= $^{32}"
env.desc = "|f_{p0}| = 16 but f_{p0} random each time, |f_{p1}| = 16"
split_file(env, file_names, [inv("tv"), inv("16_1")], f"group{i}")

i += 1
env.name = prefix_name+f"_{i}"
env.title = "f_{p0} vs f_{p1}, mask <= $^{32}"
env.desc = "|f_{p0}| = 16 but f_{p0} random each time, |f_{p1}| = 16"
split_file(env, file_names, [inv("tv"), inv("16_2")], f"group{i}")

i += 1
env.name = prefix_name+f"_{i}"
env.title = "f_{p0} vs f_{p1}, mask <= $^{32}"
env.desc = "|f_{p0}| = |f_{p1}| = 16"
split_file(env, file_names, [inv("16_1"), inv("16_2")], f"group{i}")

i += 1
env.name = prefix_name+f"_{i}"
env.title = "f_{p0} vs f_{p1}, mask <= $^{32}"
env.desc = "|f_{p0}| = 16, |f_{p1}| = 18"
split_file(env, file_names, [inv("16_1"), inv("18")], f"group{i}")

i += 1
env.name = prefix_name+f"_{i}"
env.title = "f_{p0} vs f_{p1}, mask <= $^{32}"
env.desc = "|f_{p0}| = 16, |f_{p1}| = 21"
split_file(env, file_names, [inv("16_1"), inv("21")], f"group{i}")

i += 1
env.name = prefix_name+f"_{i}"
env.title = "f_{p0} vs f_{p1}, mask <= $^{32}"
env.desc = "|f_{p0}| = 16, |f_{p1}| = 18"
split_file(env, file_names, [inv("16_2"), inv("18")], f"group{i}")

i += 1
env.name = prefix_name+f"_{i}"
env.title = "f_{p0} vs f_{p1}, mask <= $^{32}"
env.desc = "|f_{p0}| = 16, |f_{p1}| = 21"
split_file(env, file_names, [inv("16_2"), inv("21")], f"group{i}")

 
remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())

# redo with register operations


Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 4931 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4931 bytes
None
done 2024-11-20 11-54


In [ ]:
% TODO: ADD TVLA plus zero word plus different weighted words

In [ ]:
file_names = [f"group{i}" for i in range(1,8)]
change_graph(env, file_names, {"type": "time"})

In [ ]:
execute mission
# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()
for i in range(1,6):
    status = traces("file_name", 0)
split_file(["file_name"], [0, 1], "result")
remove_files(["file_name"])
generate_file("start_time", "operations")
print("done")